In [1]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

import json, requests, pickle

In [2]:
import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email import encoders

In [3]:
# Kakao Appkey 저장 파일
APP_KEY_SAVED_FILE = 'res/kakao_msg/app_key'
# Kakao Token 저장 파일
KAKAO_TOKEN_SAVED_FILE = 'res/kakao_msg/kakao_token.json'
# Naver 아이디, 비밀번호 저장 파일
NAVER_USER_DATA_SAVED_FILE = 'res/naver/account'

In [5]:
# 네이버 계정 정보 저장  
naver_user_info = dict({
    'smtp_user_id': '@naver.com',
    'smtp_user_pw': ''
})

f = open(NAVER_USER_DATA_SAVED_FILE, 'wb')
pickle.dump(naver_user_info, f)
f.close()

In [9]:
# 네이버 계정 정보 읽기
f = open(NAVER_USER_DATA_SAVED_FILE, 'rb')
naver_user_info = pickle.load(f)
f.close()

In [10]:
# 앱키 읽기
app_key = ''

f = open(APP_KEY_SAVED_FILE, 'rb')
app_key = pickle.load(f)
f.close()

In [26]:
# 토큰 발급
url = "https://kauth.kakao.com/oauth/token"

data = {
    "grant_type" : "authorization_code",
    "client_id" : "",
    "redirect_uri" : "https://localhost.com",
    "code" : ""
}

response = requests.post(url, data=data)

if response.status_code != 200:
    print("Error!")
else:
    tokens = response.json()
    print(tokens)

Error!


In [19]:
# 토큰 저장, 읽기
def save_tokens(filename, tokens):
    with open(filename, 'w') as fp:
        json.dump(tokens, fp)
        
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

save_tokens(KAKAO_TOKEN_SAVED_FILE, tokens)

In [12]:
def send_email(smtp_info, naver_user_info, msg):
    with smtplib.SMTP(smtp_info['smtp_server'], smtp_info['smtp_port']) as server:
        # TLS 보안 설정 및 시작(보안 정책상 필수)
        server.starttls()
        
        # 로그인
        server.login(naver_user_info['smtp_user_id'], naver_user_info['smtp_user_pw'])
        
        # 이메일 전송
        # 이메일 전송 시 반드시 as_string()으로 변환
        res = server.sendmail(msg['from'], msg['to'], msg.as_string())
        
        if not res:
            print('이메일 전송 성공')
        else:
            print(res)

In [13]:
def make_multimsg(msg_dict):
    multi = MIMEMultipart(_subtype = 'mixed')
    
    for key, value in msg_dict.items():
        # text 형식인 경우
        if key == 'text':
            with open(value['filename'], encoding='utf-8') as fp:
                msg = MIMEText(fp.read(), _subtype = value['subtype'])
        
        # image 형식인 경우
        elif key == 'image':
            with open(value['filename'], 'rb') as fp:
                msg = MIMEImage(fp.read(), _subtype = value['subtype'])
                
        # audio, video 형식은 설정x
        elif key == 'audio':
            print('음원은 지원하지 않습니다.')
        elif key == 'video':
            print('영상은 지원하지 않습니다.')
            
        # 그 외의 형식    
        else:
            with open(value['filename'], 'rb') as fp:
                msg = MIMEBase(value['maintype'], _subtype = value['subtype'])
                msg.set_payload(fp.read())
                encoders.encode_base64(msg)
        
        # 파일 이름은 첨부파일 제목
        msg.add_header('Content-Disposition', 'attachment',
                      filename = os.path.basename(value['filename']))
        
        # 파일 첨부
        multi.attach(msg)
        
    return multi

In [25]:
import datetime

app = Flask(__name__)
CORS(app)

@app.route('/orderTest', methods = ['GET', 'POST'])
def orderTest():
    params = json.loads(request.get_data(), encoding='utf-8')
    
    if len(params) == 0:
        return jsonify('No Parameter')
    
    params_string = ''
    
    values = []
    
    for key in params.keys():
        values.append(params[key])
        
    name = values[0]
    cost = values[1]
    
    print('이름: ', name, ', 가격: ', cost)
    
    tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)

    # 카톡 메세지 보내기
    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

    # request parameter
    headers = {
        "Authorization" : "Bearer " + tokens['access_token']
    }
    
    message = '이름: {0}, 가격: {1}'.format(name, cost)

    data = {
        "template_object" : json.dumps({
            "object_type" : "text",
            "text" : message,
            "link" : {
                "web_url" : "http://localhost:8080"
            }
        })
    }

    # 메세지 전송
    response = requests.post(url, headers=headers, data=data)
    print(response.status_code)
        
    # 메일 전송하기
    smtp_info = dict({
        'smtp_server': 'smtp.naver.com',
        'smtp_port': 587
    })

    msg_dict = {
        'image': {
            'maintype': 'image',
            'subtype': 'jfif',
        },
    }
    
    # 결제 내용(이메일) 추가하기
    now = datetime.datetime.now()
    now_time = now.strftime('%Y-%m-%d %H:%M:%S')
    
    title = name + '님 ' + now_time + ' 결제하신 내역입니다.'
    content = '구매하신 물품은 ... 이며 결제하신 금액은 ' + str(cost) + '원 입니다.' 
    sender = naver_user_info['smtp_user_id']
    receiver = naver_user_info['smtp_user_id']

    msg = MIMEText(_text = content, _charset = 'utf-8')

    # 특정위치 지정하기
    msg_dict['image']['filename'] = 'email/보노보노.jfif'
    
    # 파일 첨부
    multi = make_multimsg(msg_dict)
    multi['Subject'] = title
    multi['From'] = sender
    multi['To'] = receiver
    multi.attach(msg)

    send_email(smtp_info, naver_user_info, multi)

    return jsonify(params_string)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
<ipython-input-25-a42565a42a50>:8: DeprecationWarning: 'encoding' is ignored and deprecated. It will be removed in Python 3.9
  params = json.loads(request.get_data(), encoding='utf-8')


이름:  나야나 , 가격:  200000
200


127.0.0.1 - - [07/Oct/2021 18:12:36] "POST /orderTest HTTP/1.1" 200 -


이메일 전송 성공
